安装依赖

    pip install -U langchain-openai
    pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

设置 OpenAI KEY

    export OPENAI_API_KEY="sk-xxx"

In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
import getpass
import os

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [3]:
model = init_chat_model("openai:gpt-4o-mini", temperature=0)
model.invoke("你是谁？")

AIMessage(content='我是一个人工智能助手，旨在回答问题和提供信息。如果你有任何问题或需要帮助，随时可以问我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 10, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3uR47OPpR3GBUgZ5aS7oX36blAm2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--6bd1de8f-84a4-4c60-bbab-4332a96f8f8d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 28, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
# Create the agent
memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [5]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "你好，我是蛙蛙，我住在中国烟台。",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

你好，我是蛙蛙，我住在中国烟台。
================================== Ai Message ==================================

你好，蛙蛙！很高兴认识你！烟台是一个美丽的城市，有着丰富的海洋资源和美丽的风景。你在烟台生活得怎么样？有什么想聊的或者需要帮助的呢？


In [6]:
input_message = {
    "role": "user",
    "content": "我住的城市今天天气怎么样？",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我住的城市今天天气怎么样？
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_fnT0vUcASc6ZXAQev6eE1TrC)
 Call ID: call_fnT0vUcASc6ZXAQev6eE1TrC
  Args:
    query: 烟台今天天气
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "烟台今天天气", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://tianqi.moji.com/weather/china/shandong/yantai", "title": "烟台市天气预报", "content": "烟台市今天实况：25度多云，湿度：78%，南风：2级。白天：32度,晴。 夜间：多云，24度，潮湿闷热，墨迹天气建议您穿着吸湿排汗的夏装，棉麻质地的短打扮是不错的选择。", "score": 0.8851659, "raw_content": null}, {"url": "http://www.nmc.cn/publish/forecast/ASD/yantai.html", "title": "烟台-天气预报", "content": "11:00 · 28.2℃. 7.8m/s. 北风. 1001.8hPa ; 14:00 · 29.8℃. 7.9m/s. 东北风. 1001.4hPa ; 17:00 · 28.2℃. 7m/s. 北风. 1001.6hPa.", "score": 0.755078, "raw_content": null}], "respo